In [60]:
from datetime import datetime 
import re
import sqlite3
import sys
import numpy as np
import pandas as pd

import messages from textfile. Parse date,author, text (multiline) and message type. The textfile is created by exporting and emailing the chat conversation from the whatapp chat settings

In [61]:
with open('messages.txt', encoding='utf-8') as file:
    content = file.readlines()
    data = []
    i=0
    while i<len(content):
        message = content[i]
        j=0

        while (i+j+1)<len(content) and not re.match('\d{2}/\d{2}/\d{4}, \d{2}:\d{2} - ',content[i+j+1]):
            messagecont = content[i+j+1]
            #print("adding multiline {}".format(messagecont))
            message = "{} {}".format(message,messagecont)
            i=i+1

        i=i+j

        p = message.find(' - ')
        date = message[0:p-1].strip()
        message = message[p+2:].strip()


        dt = datetime.strptime(date,'%d/%m/%Y, %H:%M')
        p = message.find(':')
        if (p==-1):
            system = True
            author = "system"
            message = message
        else: 
            system = False
            author = message[0:p].strip()
            message = message[p+1:].strip()

        #print("On {} {} wrote {}".format(dt,author,message))
        params = [dt,author,message,system]
        data.append(params)
        #c.execute("INSERT INTO messages (postdate,author,message,system) VALUES (?,?,?,?)",params)


        i=i+1

In [62]:
df = pd.DataFrame(data, columns=['postdate','author','text','systemmessage']  )


Who are the 10 most frequent chaters off all times?

In [63]:
df.groupby('author').count().sort_values('text',ascending=False).nlargest(10,'text')

,postdate,text,systemmessage
author,,,
Timo,1288,1288,1288
Leandro Vb,740,740,740
Mirja Vb,658,658,658
system,518,518,518
Junior Vb,412,412,412
Desi Vb,375,375,375
Victor Barbe Vb,298,298,298
Antho Vb,296,296,296
Nico Vb,292,292,292


In [68]:
df.loc[:,'systemmessage']

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12       False
13       False
14       False
15        True
16        True
17       False
18        True
19       False
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28       False
29        True
         ...  
11768    False
11769     True
11770    False
11771    False
11772    False
11773    False
11774    False
11775    False
11776    False
11777    False
11778    False
11779    False
11780    False
11781    False
11782    False
11783    False
11784    False
11785    False
11786    False
11787    False
11788    False
11789    False
11790    False
11791    False
11792    False
11793    False
11794    False
11795    False
11796    False
11797    False
Name: systemmessage, Length: 11798, dtype: bool